## Table of Contents

<div>
        <a href="#item1">1. Business Statement</a>
        <br>
        <a href="##item2">2. Data Source</a>
        <br>
        <a href="#item3">3. Analyze Data</a>
        <br>
        <a href="#item4">4. Visualise Data</a>
        <br>
        <a href="#item5">5. Conclusions</a>    
        <br>
    </ol>
</div>

## 1. Business Statement

A new fitness chain would like to expand in Denmark. Opening the first Gym is going to give the stakeholder an idea on how profitable is their business model and how it will be received might dictate future plans for expansion on the Danish Market.

Data Analysis should be used to find the best location considering a few criteria:

Proximity to high traffic areas
A certain distance to existing fitness centers.
A location with lower density of Gym facilities.




## 2. Data Source

The main Data Source will be Foursquare API for identifying existing gym and mapping them over the Central Copenhagen zip codes.
Data from Denmark Statistics Institute will be used to identify higher density areas and popular venues.
People are more likely to choose a gym close to a Shopping Center then in a remote area so this should be factored into account.

By knowing existing Gym and their area of coverage, by applying unsupervised learning techniques it will be determined the optimum location for opening a new fitness center.

Using Foursquare let's fetch 

In [1]:
### A. Let's import the Fitness Gyms that are registered for Copenhagen from Foursquare.

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
# import k-means from clustering stage
from sklearn.cluster import KMeans

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
from pandas.io.json import json_normalize
from folium.plugins import HeatMap


print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.0               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


bran

In [2]:
## Adding Foursquare Credentials:


CLIENT_ID = 'FZF2SWDMAVWXMFYAXUYYDKCLZYHQX0WESCXZFEU5ZNQ2DLLL' # your Foursquare ID
CLIENT_SECRET = 'WSRM2GTQKNXLREEGDXDR4ZWE4LNVILAKOPPVRPVY5H2OL3AO' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#Let's define the user agent:

address = 'Copenhagen'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

# Let's define the search based on location

search_query = 'Fitness'

print(search_query + ' .... OK!')


radius = 10000 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API

# Let's print the url to fetch the venues
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
print(url)
# Let's make a request and print the results:

results = requests.get(url).json()

results

Your credentails:
CLIENT_ID: FZF2SWDMAVWXMFYAXUYYDKCLZYHQX0WESCXZFEU5ZNQ2DLLL
CLIENT_SECRET:WSRM2GTQKNXLREEGDXDR4ZWE4LNVILAKOPPVRPVY5H2OL3AO
55.6867243 12.5700724
Fitness .... OK!
https://api.foursquare.com/v2/venues/search?client_id=FZF2SWDMAVWXMFYAXUYYDKCLZYHQX0WESCXZFEU5ZNQ2DLLL&client_secret=WSRM2GTQKNXLREEGDXDR4ZWE4LNVILAKOPPVRPVY5H2OL3AO&ll=55.6867243,12.5700724&v=20180604&query=Fitness&radius=10000&limit=100


{'meta': {'code': 200, 'requestId': '5eb0b56a0cc1fd001b46578e'},
 'response': {'venues': [{'id': '4b9bc30bf964a520402236e3',
    'name': 'Fitness World',
    'location': {'address': 'Købmagergade 48',
     'lat': 55.681079503234926,
     'lng': 12.57663229853562,
     'labeledLatLngs': [{'label': 'display',
       'lat': 55.681079503234926,
       'lng': 12.57663229853562}],
     'distance': 751,
     'postalCode': '1150',
     'cc': 'DK',
     'city': 'København K',
     'state': 'Region Hovedstaden',
     'country': 'Danmark',
     'formattedAddress': ['Købmagergade 48', '1150 København K', 'Danmark']},
    'categories': [{'id': '4bf58dd8d48988d175941735',
      'name': 'Gym / Fitness Center',
      'pluralName': 'Gyms or Fitness Centers',
      'shortName': 'Gym / Fitness',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1588639200',
    'hasPerk': False},
   {'id': '4d2ac6eb8292

## 3. Analyze Data

Analyze Data

#### Clear the Data and Transform it in a Dataframe:

In [3]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

dataframe.shape

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe_filtered
    
# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]


In [4]:
#Let's see the list of gyms:

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Fitness World,Gym / Fitness Center,Købmagergade 48,DK,København K,Danmark,NaN,751,"[Købmagergade 48, 1150 København K, Danmark]","[{'label': 'display', 'lat': 55.68107950323492...",55.681080,12.576632,NaN,1150,Region Hovedstaden,4b9bc30bf964a520402236e3
1,Fitness World,Gym,Worsaaesvej 17,DK,Frederiksberg,Danmark,NaN,1182,"[Worsaaesvej 17, 1972 Frederiksberg, Danmark]","[{'label': 'display', 'lat': 55.68315419305056...",55.683154,12.552322,NaN,1972,Region Hovedstaden,4d2ac6eb8292236adf772ebb
2,Fitness World,Gym / Fitness Center,Esromgade 15,DK,København N,Danmark,NaN,2209,"[Esromgade 15, 2200 København N, Danmark]","[{'label': 'display', 'lat': 55.69739900482128...",55.697399,12.540378,NaN,2200,Nørrebro,5549c800498ef76f3c5f958d
3,Fitness World,Gym / Fitness Center,Gasværksvej 16,DK,København V,Danmark,NaN,1980,"[Gasværksvej 16, 1656 København V, Danmark]","[{'label': 'display', 'lat': 55.6706799, 'lng'...",55.670680,12.556421,NaN,1656,Region Hovedstaden,4b3c9071f964a520298525e3
4,Fitness World,Gym / Fitness Center,Tagensvej 129,DK,København N,Danmark,NaN,2602,"[Tagensvej 129, 2200 København N, Danmark]","[{'label': 'display', 'lat': 55.70516915274733...",55.705169,12.544592,NaN,2200,NaN,5a5cc156ad17890e6bbd35ef
5,Fitness World,Gym / Fitness Center,Jagtvej 113-115,DK,København N,Danmark,NaN,1621,"[Jagtvej 113-115, 2200 København N, Danmark]","[{'label': 'display', 'lat': 55.6965957, 'lng'...",55.696596,12.551058,NaN,2200,NaN,5020058ae4b061ae1add2500
6,Fitness World,Gym / Fitness Center,Århusgade 102,DK,København Ø,Danmark,NaN,2478,"[Århusgade 102, 2100 København Ø, Danmark]","[{'label': 'display', 'lat': 55.7067226, 'lng'...",55.706723,12.587448,NaN,2100,Region Hovedstaden,4b14c89cf964a52082a623e3
7,Fitness dk,Gym,Amager Blvd. 70,DK,København S,Danmark,NaN,2415,"[Amager Blvd. 70, 2300 København S, Danmark]","[{'label': 'display', 'lat': 55.66709616012887...",55.667096,12.586491,NaN,2300,NaN,4b8eac83f964a5201b3033e3
8,Fitness World,Gym / Fitness Center,Lyongade 23-25,DK,København S,Danmark,NaN,3774,"[Lyongade 23-25, 2300 København S, Danmark]","[{'label': 'display', 'lat': 55.66144308636816...",55.661443,12.610136,NaN,2300,Region Hovedstaden,4b44e3bbf964a52066ff25e3
9,Fitness World,Gym / Fitness Center,Danneskiold-Samsøes Allé 20,DK,København,Danmark,NaN,2296,"[Danneskiold-Samsøes Allé 20, 1434 København, ...","[{'label': 'display', 'lat': 55.6780169, 'lng'...",55.678017,12.603238,NaN,1434,Region Hovedstaden,4b3f6ffcf964a520d4a625e3


## 4. Visualize Data


In [5]:
# Plotting the Gym addresses on the Map

In [6]:
# Let's see the gyms on a map:

print('Folium installed')

dataframe_filtered['lat'] = dataframe_filtered['lat'].astype(float)
dataframe_filtered['lng'] = dataframe_filtered['lng'].astype(float)


venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 


#folium.features.CircleMarker(
#    [latitude, longitude],
#    radius=30,
#    color='red',
#    popup='Copenhagen',
#    fill = True,
#    fill_color = 'red',
#    fill_opacity = 0.6
#).add_to(venues_map)


for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
       popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.1
    ).add_to(venues_map)

heat_data = [[row['lat'],row['lng']] for index, row in dataframe_filtered.iterrows()]

venues_map.add_child(HeatMap(heat_data, radius = 75, gradient={.01: 'blue', .3: 'lime', 1: 'red'}))

    
# display map
venues_map

Folium installed


## 5. Conclusions

Conclusions

Based on the above, the locations and the areas of influence can be adjusted 
considering the population density and a better picture of training facilities 
per person can be obtained.



A gym in the red area is not a guarantee that it will not perform, 
but probably the marketing efforts will have to be higher to make 
the gym known and to attract members from other gyms.



The competition is already high if there are 3-4 gyms clustered 
in an area so free offers, no signup fee, personalised training 
advice and programs could be considered as a marketing strategy.
